In [4]:
import pandas as pd
import numpy as np
from SFFirePredLibraries import preprocessInput


In [5]:
df = pd.read_csv('Fire_Incidents.tsv', sep='\t')
df.describe()


C:\Users\Anjali\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (11,12,19,37,44,58) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Incident Number,Exposure Number,Call Number,Zipcode,Suppression Units,Suppression Personnel,EMS Units,EMS Personnel,Other Units,Other Personnel,...,Civilian Fatalities,Civilian Injuries,Number of Alarms,Floor of Fire Origin,Number of floors with minimum damage,Number of floors with significant damage,Number of floors with heavy damage,Number of floors with extreme damage,Number of Sprinkler Heads Operating,Supervisor District
count,4.930610e+05,493061.000000,4.930610e+05,420240.000000,493061.000000,493061.000000,493061.000000,493061.000000,493061.000000,493061.000000,...,493061.000000,493061.000000,0.0,8151.000000,6610.000000,6599.000000,6593.000000,6596.000000,6742.000000,420220.000000
mean,1.079609e+07,0.000045,1.091767e+08,94113.485354,2.536072,9.124415,0.176471,0.326905,0.034582,0.063317,...,0.000061,0.001158,NaN,2.275426,0.723298,0.155781,0.082967,0.039418,0.081282,5.786017
std,4.726111e+06,0.009229,4.716861e+07,10.174619,8.896839,22.983902,0.783586,1.196615,1.449118,1.518516,...,0.009229,0.065608,NaN,5.760013,0.510876,0.438704,0.339013,0.266318,0.431558,2.729472
min,3.000001e+06,0.000000,3.001000e+07,94102.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,NaN,-3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,7.024104e+06,0.000000,7.082034e+07,94105.000000,1.000000,4.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,NaN,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000
50%,1.103297e+07,0.000000,1.109903e+08,94110.000000,2.000000,9.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,NaN,2.000000,1.000000,0.000000,0.000000,0.000000,0.000000,6.000000
75%,1.505559e+07,0.000000,1.514621e+08,94121.000000,3.000000,10.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,NaN,2.000000,1.000000,0.000000,0.000000,0.000000,0.000000,8.000000
max,1.904402e+07,5.000000,1.910338e+08,94158.000000,3333.000000,5960.000000,150.000000,312.000000,1000.000000,1000.000000,...,3.000000,24.000000,NaN,105.000000,5.000000,7.000000,5.000000,6.000000,20.000000,11.000000


In [6]:
df_feature = preprocessInput(df)
filter_df = df_feature[df_feature['Suppression Personnel'] <= 100]
filter_df.shape
filter_df['Suppression Personnel'].value_counts()
df_Y = filter_df['Suppression Personnel']
filter_df = filter_df.drop(['Suppression Personnel'],axis=1)
df_Y.shape


C:\Users\Anjali\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3185: DtypeWarning: Columns (11,12,19,37,44,58) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


(492927,)

In [7]:
from scipy.sparse import csr_matrix

def dataframetoCSRmatrix(df):
    nrows = len(df)
    nc = len(df.columns)
    idx = {}
    tid = 0
    nnz = nc * nrows
    
    cols= df.columns
    
    for col in cols:
        df[col] = df[col].apply(str)
        for name in df[col].unique():
            idx[col+name] = tid
            tid += 1
    
    ncols = len(idx)
    
    ind = np.zeros(nnz, dtype=np.int)
    val = np.zeros(nnz, dtype=np.int)
    ptr = np.zeros(nrows+1, dtype=np.int)
    
    i=0
    n=0
    
    for index,row in df.iterrows():
        for j,col in enumerate(cols):
            ind[j+n] = idx[col+row[col]]
            val[j+n] = 1
        ptr[i+1] = ptr[i] + nc
        n += nc
        i += 1
    
    mat = csr_matrix((val,ind,ptr), shape=(nrows,ncols), dtype=np.int)
    mat.sort_indices()   
    
    return mat

In [8]:
mat1 = dataframetoCSRmatrix(filter_df)

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import explained_variance_score
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import median_absolute_error
from sklearn.metrics import mean_squared_log_error

In [10]:
df_feature_train, df_feature_test, df_y_train, df_y_test = train_test_split(mat1, df_Y, test_size = 0.2, random_state = 0)


rfreg = RandomForestRegressor(n_estimators= 10,max_depth=1, random_state=42)
rfreg.fit(df_feature_train, df_y_train)


predicted_y_rf = rfreg.predict(df_feature_test)


set(predicted_y_rf)



{8.688614182478034, 30.31887310100881}

In [11]:
mat1.shape

(492927, 644)

In [12]:
df_Y.values.shape

(492927,)

In [13]:

print("r2_score: ",r2_score(df_y_test, predicted_y_rf))
print("explained_variance_score: ",explained_variance_score(df_y_test, predicted_y_rf))
print("mean_absolute_error: ",mean_absolute_error(df_y_test, predicted_y_rf))
print("mean_squared_error: ",mean_squared_error(df_y_test, predicted_y_rf))
print("median_absolute_error: ",median_absolute_error(df_y_test, predicted_y_rf))
print("mean_squared_log_error: ",mean_squared_log_error(df_y_test, predicted_y_rf))


r2_score:  0.136132331423771
explained_variance_score:  0.13613351516637073
mean_absolute_error:  3.8622397381492335
mean_squared_error:  37.872875841645175
median_absolute_error:  2.688614182478034
mean_squared_log_error:  0.27561720179386173


In [14]:
from sklearn.externals import joblib
joblib.dump(rfreg, 'rforest.pkl')

['rforest.pkl']